In [15]:
%matplotlib inline
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dropout, Flatten, Dense, Activation, Conv2D, MaxPooling2D
from keras.models import Sequential
import numpy as np
from pak.datasets.CUHK03 import cuhk03
import matplotlib.pyplot as plt
from keras.optimizers import SGD
import json
from pprint import pprint
Settings = json.load(open('../prototyping/settings.txt'))
pprint(Settings)
from pak.datasets.CUHK03 import cuhk03

root = '/home/data'
target_w, target_h = 112, 112
data = cuhk03(root, target_w=target_w, target_h=target_h)
X, Y = data.get_labeled()
X, Y = data.get_detected()

{'data_root': '/home/data',
 'deepmatch': '/home/deepmatching_1.2.2_c++/deepmatching-static'}
/home/data/cuhk03_release found :)
found memmaped /home/data/cuhk03_release/labeled112x112.npy
found memmaped /home/data/cuhk03_release/detected112x112.npy


In [13]:
vgg_model = VGG16(input_shape=(112, 112, 3), weights='imagenet', include_top=False)


train_upper_layers = True
model = Sequential()
block1_conv1 = Conv2D(64, (3, 3), input_shape=(112,112,6), 
                 padding='same', name='block1_conv1', 
                      trainable=train_upper_layers)
model.add(block1_conv1)
model.add(Conv2D(64, (3,3), padding='same', name='block1_conv2', 
                 trainable=train_upper_layers))
model.add(MaxPooling2D(pool_size=(2,2), name='block1_pool'))

model.add(Conv2D(128, (3,3), padding='same', name='block2_conv1', 
                 trainable=train_upper_layers))
model.add(Conv2D(128, (3,3), padding='same', name='block2_conv2', 
                 trainable=train_upper_layers))
model.add(MaxPooling2D(pool_size=(2,2), name='block2_pool'))

model.add(Conv2D(256, (3,3), padding='same', name='block3_conv1', 
                 trainable=train_upper_layers))
model.add(Conv2D(256, (3,3), padding='same', name='block3_conv2', 
                 trainable=train_upper_layers))
model.add(Conv2D(256, (3,3), padding='same', name='block3_conv3', 
                 trainable=train_upper_layers))
model.add(MaxPooling2D(pool_size=(2,2), name='block3_pool'))

model.add(Conv2D(512, (3,3), padding='same', name='block4_conv1', 
                 trainable=train_upper_layers))
model.add(Conv2D(512, (3,3), padding='same', name='block4_conv2', 
                 trainable=train_upper_layers))
model.add(Conv2D(512, (3,3), padding='same', name='block4_conv3', 
                 trainable=train_upper_layers))
model.add(MaxPooling2D(pool_size=(2,2), name='block4_pool'))

model.add(Conv2D(512, (3,3), padding='same', name='block5_conv1'))
model.add(Conv2D(512, (3,3), padding='same', name='block5_conv2'))
model.add(Conv2D(512, (3,3), padding='same', name='block5_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), name='block5_pool'))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# --- set the fixed weights ---
W, bias = vgg_model.layers[1].get_weights()
W = np.concatenate((W,W),axis=2)
block1_conv1.set_weights([W, bias])
# -----------------------------

for i in range(1, len(vgg_model.layers)-1):
    model.layers[i].set_weights(vgg_model.layers[i+1].get_weights())

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 112, 112, 64)      3520      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 112, 112, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 56, 56, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 56, 56, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 56, 56, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 28, 28, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 28, 28, 256)       295168    
__________